# Create Dataset from Wiki Tables

## Import Packages

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime as dt
import statistics as stats

## Functions

In [2]:
# read_wiki pulls in wiki data from URL and returns the soup
def read_wiki(url, params):
    req = requests.get(url, params)
    wikisoup = BeautifulSoup(req.content, "html.parser")
    return wikisoup

# get_genres parses the soup to get the list of genres
def get_genres(soup):
    genre_soup = soup.find_all(class_="mw-headline")
    genre_ls = []
    for genre in genre_soup:
        content = genre.text.strip()
        genre_ls.append(content)
    return genre_ls

# remove_genres takes out the genres that don't have tables
def remove_genres(genre_ls, remove_ls):
    for i in remove_ls:
        genre_ls.remove(i)
    return genre_ls

# get_tables parses out each genres table from the wiki soup
def get_tables(soup, genre_ls):
    tables_soup = soup.find_all("table", \
                                class_=["wikitable sortable", \
                                        "wikitable sortable plainrowheaders", \
                                        "wikitable plainrowheaders sortable"])
    tables_df = pd.DataFrame()
    for i in range(0, len(genre_ls)):
        table = tables_soup[i]
        df = pd.read_html(str(table))
        df = pd.DataFrame(df[0])
        tables_df = pd.concat([tables_df, df], ignore_index=True, sort=False)
    return tables_df

## Data Setup

In [3]:
# Request requirements for each url
PARAMS = {"client":"firefox-b-d", "User-agent":"kalesreg"}
nflx_cont_url = "https://en.wikipedia.org/wiki/List_of_Netflix_original_programming"
nflx_end_url = "https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming"
hulu_url = "https://en.wikipedia.org/wiki/List_of_Hulu_original_programming"
amzn_url = "https://en.wikipedia.org/wiki/List_of_Amazon_Prime_Video_original_programming"
apl_url = "https://en.wikipedia.org/wiki/List_of_Apple_TV%2B_original_programming"
prmt_url = "https://en.wikipedia.org/wiki/List_of_Paramount%2B_original_programming"
dis_url = "https://en.wikipedia.org/wiki/List_of_Disney%2B_original_programming"
hbo_url = "https://en.wikipedia.org/wiki/List_of_HBO_Max_original_programming"
#url_ls = [nflx_cont_url, nflx_end_url, hulu_url, amzn_url, apl_url, prmt_url, dis_url, hbo_url]

## Netflix Data

In [77]:
# Netflix Current Data
nflx_cont_soup = read_wiki(nflx_cont_url, PARAMS)
nflx_cont_genres = get_genres(nflx_cont_soup)
#print(nflx_cont_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = nflx_cont_genres.index("Upcoming original programming")
nflx_cont_genres = nflx_cont_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
nflx_cont_remove = ["Animation", "Non-English language scripted", "Unscripted", "Specials"]
nflx_cont_genres = remove_genres(nflx_cont_genres, nflx_cont_remove)
#print(nflx_cont_genres)
nflx_cont_df = get_tables(nflx_cont_soup, nflx_cont_genres)
#display(nflx_cont_df)
nflx_cont_df["Cur. service"] = "Netflix"
nflx_cont_df = nflx_cont_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime"]]
display(nflx_cont_df)

# Netflix Ended Data
nflx_end_soup = read_wiki(nflx_end_url, PARAMS)
nflx_end_genres = get_genres(nflx_end_soup)
#print(nflx_end_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = nflx_end_genres.index("Notes")
nflx_end_genres = nflx_end_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
nflx_end_remove = ["Animation", "Non-English language scripted", "Unscripted", "Specials", "Regional original programming", "Animation", "Non-English language scripted", "Unscripted"]
nflx_end_genres = remove_genres(nflx_end_genres, nflx_end_remove)
#print(nflx_end_genres)
nflx_end_df = get_tables(nflx_end_soup, nflx_end_genres)
#display(nflx_end_df)
for x in range(0, len(nflx_end_df)):
    if pd.isnull(nflx_end_df.loc[x, "Premiere"]):
        nflx_end_df.loc[x, "Premiere"] = nflx_end_df.loc[x, "Release date"]
nflx_end_df["Seasons"] = nflx_end_df[['Seasons', 'Episodes']].stack().groupby(level=0).agg(' '.join)
nflx_end_df["Status"] = "Ended"
nflx_end_df["Cur. service"] = "Netflix"
nflx_end_df = nflx_end_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime"]]
display(nflx_end_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime
0,Netflix,Stranger Things,Science fiction horror,"July 15, 2016","4 seasons, 34 episodes",Renewed for final season[1],NaN,42–150 min
1,Netflix,The Crown,Historical drama,"November 4, 2016","5 seasons, 50 episodes",Renewed for final season[2],NaN,47–61 min
2,Netflix,The Umbrella Academy,Superhero action,"February 15, 2019","3 seasons, 30 episodes",Renewed for final season[3],NaN,40–60 min
3,Netflix,Virgin River,Romantic drama,"December 6, 2019","4 seasons, 42 episodes",Renewed[4],NaN,39–49 min
4,Netflix,The Witcher,Fantasy drama,"December 20, 2019","2 seasons, 16 episodes",Season 3 due to premiere in 2023[5] Renewed fo...,NaN,47–67 min
...,...,...,...,...,...,...,...,...
385,Netflix,The Nutty Boy,Animated comedy,"October 12, 2022","1 season, 9 episodes",Pending,Portuguese,10–13 min
386,Netflix,The Final Score,Drama,"November 2, 2022","1 season, 6 episodes",Pending,Spanish,43–58 min
387,Netflix,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release
388,Netflix,Brown and Friends[208],Animated comedy,"December 29, 2022",TBA,Pending,No spoken language,TBA


,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime
0,Netflix,House of Cards,Political drama,"February 1, 2013","6 seasons, 73 episodes",Ended,NaN,42–59 min
1,Netflix,Hemlock Grove,Horror/thriller,"April 19, 2013","3 seasons, 33 episodes",Ended,NaN,45–58 min
2,Netflix,Orange Is the New Black,Comedy drama,"July 11, 2013","7 seasons, 91 episodes",Ended,NaN,50–92 min
3,Netflix,Marco Polo,Historical drama,"December 12, 2014","2 seasons, 20 episodes",Ended,NaN,48–65 min
4,Netflix,Bloodline,Thriller,"March 20, 2015","3 seasons, 33 episodes",Ended,NaN,48–68 min
...,...,...,...,...,...,...,...,...
1099,Netflix,Mystery Science Theater 3000 (seasons 11–12),Comic science fiction,"April 14, 2017","2 seasons, 20 episodes",Ended,English,86–94 min
1100,Netflix,Slasher (season 2),Horror anthology series,"October 17, 2017","1 season, 8 episodes",Ended,English,46–53 min
1101,Netflix,EastSiders (seasons 3–4),Black comedy,"November 28, 2017","2 seasons, 12 episodes",Ended,English,25 min
1102,Netflix,The Mafia Dolls (season 2),Telenovela,"March 15, 2019","1 season, 60 episodes",Ended,Spanish,42–61 min


## Hulu Data

In [36]:
# Hulu Data
hulu_soup = read_wiki(hulu_url, PARAMS)
hulu_genres = get_genres(hulu_soup)
#print(hulu_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = hulu_genres.index("Upcoming original programming")
#print(end)
hulu_genres = hulu_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
hulu_remove = ["Original programming", "Animation", "Unscripted", "Hotstar"]
hulu_genres = remove_genres(hulu_genres, hulu_remove)
#print(hulu_genres)
hulu_df = get_tables(hulu_soup, hulu_genres)
#display(hulu_df)
hulu_df.rename(columns={"Length":"Runtime"}, inplace=True)
hulu_df["Cur. service"] = "Hulu"
hulu_df = hulu_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime", "Prev. network(s)"]]
display(hulu_df)

15


,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s)
0,Hulu,East Los High,Teen drama,"June 3, 2013","4 seasons, 61 episodes",Ended[3],NaN,22–24 min.,NaN
1,Hulu,11.22.63,Drama,"February 15, 2016",8 episodes,Miniseries,NaN,44–81 min.,NaN
2,Hulu,The Path,Drama,"March 30, 2016","3 seasons, 36 episodes",Ended[4],NaN,45–56 min.,NaN
3,Hulu,Freakish,Horror,"October 10, 2016","2 seasons, 20 episodes",Ended[5],NaN,22–24 min.,NaN
4,Hulu,Shut Eye,Drama,"December 7, 2016","2 seasons, 20 episodes",Ended[6],NaN,40–42 min.,NaN
...,...,...,...,...,...,...,...,...,...
173,Hulu,The Tatami Time Machine Blues,Mystery/Romantic comedy anime,"November 9, 2022",6 episodes,Miniseries,Japanese,17–32 min.,NaN
174,Hulu,Kiss Sixth Sense,Romantic fantasy drama,"November 30, 2022",12 episodes,Miniseries,Korean,62–73 min.,NaN
175,Hulu,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release,NaN
176,Hulu,Connect [68],Fantasy crime thriller,"December 7, 2022",6 episodes,Miniseries,Korean,45 min.,NaN


## Amazon Prime Data

In [9]:
# Amazon Prime Data
amzn_soup = read_wiki(amzn_url, PARAMS)
amzn_genres = get_genres(amzn_soup)
#print(amzn_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = amzn_genres.index("Upcoming original programming")
amzn_genres = amzn_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
amzn_remove = ["Original programming", "Animation", "Non-English language scripted", "Unscripted", "Regional original programming"]
amzn_genres = remove_genres(amzn_genres, amzn_remove)
#print(amzn_genres)
amzn_df = get_tables(amzn_soup, amzn_genres)
#display(amzn_df)
amzn_df = amzn_df.rename(columns={"Seasons/episodes":"Seasons", "Previous channel":"Prev. network(s)"})
amzn_df["Runtime"] = "na"
amzn_df["Cur. service"] = "Amazon Prime"
amzn_df = amzn_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime", "Prev. network(s)"]]
display(amzn_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s)
0,Amazon Prime,Bosch,Detective fiction,"February 13, 2015","7 seasons, 68 episodes",Ended[2][3],NaN,na,NaN
1,Amazon Prime,Sneaky Pete,Crime drama,"August 5, 2015","3 seasons, 30 episodes",Ended[4],NaN,na,NaN
2,Amazon Prime,Hand of God,Psychological thriller,"September 4, 2015","2 seasons, 20 episodes",Ended[5],NaN,na,NaN
3,Amazon Prime,Patriot,Crime drama,"November 5, 2015","2 seasons, 18 episodes",Ended[6],NaN,na,NaN
4,Amazon Prime,Z: The Beginning of Everything,Historical period drama,"November 5, 2015","1 season, 10 episodes",Ended[7],NaN,na,NaN
...,...,...,...,...,...,...,...,...,...
370,Amazon Prime,The Outlaws,Crime comedy thriller,"October 25, 2021","2 seasons, 12 episodes",Pending,English,na,NaN
371,Amazon Prime,Stories to Stay Awake,Horror anthology,"November 5, 2021",4 episodes,Miniseries,Spanish,na,NaN
372,Amazon Prime,Der Lack ist ab (season 4–5),Comedy,"December 19, 2017","2 seasons, 20 episodes",Ended,German,na,NaN
373,Amazon Prime,Pastewka (season 8–10),Comedy,"January 26, 2018","3 seasons, 30 episodes",Ended,German,na,NaN


## Apple TV+ Data

In [17]:
# Apple TV+ Data
apl_soup = read_wiki(apl_url, PARAMS)
apl_genres = get_genres(apl_soup)
#print(apl_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = apl_genres.index("Upcoming original programming")
apl_genres = apl_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
apl_remove = ["Original programming", "Animation", "Unscripted"]
apl_genres = remove_genres(apl_genres, apl_remove)
#print(apl_genres)
apl_df = get_tables(apl_soup, apl_genres)
#display(apl_df)
apl_df["Language"] = np.NaN
apl_df["Prev. network(s)"] = np.NaN
apl_df["Cur. service"] = "Apple TV+"
apl_df = apl_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime", "Prev. network(s)"]]
display(apl_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s)
0,Apple TV+,For All Mankind,Alternate history,"November 1, 2019","3 seasons, 30 episodes",Renewed[9],NaN,46–82 min.,NaN
1,Apple TV+,The Morning Show,Drama,"November 1, 2019","2 seasons, 20 episodes",Renewed[10],NaN,50–66 min.,NaN
2,Apple TV+,See,Science fiction,"November 1, 2019","3 seasons, 24 episodes",Ended[11],NaN,42–62 min.,NaN
3,Apple TV+,Servant,Psychological horror,"November 28, 2019","3 seasons, 30 episodes","Final season due to premiere on January 13, 20...",NaN,25–36 min.,NaN
4,Apple TV+,Truth Be Told,Legal drama,"December 6, 2019","2 seasons, 18 episodes",Season 3 due to premiere in 2022[13][14],NaN,39–51 min.,NaN
...,...,...,...,...,...,...,...,...,...
98,Apple TV+,The Problem with Jon Stewart,News/talk show,"September 30, 2021","2 seasons, 11 episodes",Pending,NaN,41–57 min.,NaN
99,Apple TV+,Tehran,Spy thriller,"September 25, 2020","2 seasons, 16 episodes",Pending,NaN,37–51 min.,NaN
100,Apple TV+,Losing Alice,Erotic thriller,"January 22, 2021","1 season, 8 episodes",Ended,NaN,42–55 min.,NaN
101,Apple TV+,Calls,Mystery thriller,"March 19, 2021","1 season, 9 episodes",Ended,NaN,13–20 min.,NaN


## Paramount+ Data

In [12]:
# Paramount+ Data
prmt_soup = read_wiki(prmt_url, PARAMS)
prmt_genres = get_genres(prmt_soup)
#print(prmt_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = prmt_genres.index("Upcoming original programming")
prmt_genres = prmt_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
prmt_remove = ["Original programming", "Animation", "Unscripted", "Regional original programming", "Non-English language"]
prmt_genres = remove_genres(prmt_genres, prmt_remove)
#print(prmt_genres)
prmt_df = get_tables(prmt_soup, prmt_genres)
#display(prmt_df)
prmt_df["Cur. service"] = "Paramount+"
prmt_df = prmt_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime", "Prev. network(s)"]]
display(prmt_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s)
0,Paramount+,The Good Fight,Legal drama,"February 19, 2017","6 seasons, 60 episodes",Ended,NaN,40–59 min.,NaN
1,Paramount+,Star Trek: Discovery,Science fiction,"September 24, 2017","4 seasons, 55 episodes",Season 5 due to premiere in 2023[1][2],NaN,37–65 min.,NaN
2,Paramount+,Strange Angel,Historical period drama,"June 14, 2018","2 seasons, 17 episodes",Ended[3],NaN,46–54 min.,NaN
3,Paramount+,One Dollar,Mystery thriller,"August 30, 2018","1 season, 10 episodes",Ended[4],NaN,50–57 min.,NaN
4,Paramount+,Star Trek: Short Treks,Science fiction anthology,"October 4, 2018","2 seasons, 10 episodes",Ended,NaN,8–18 min.,NaN
...,...,...,...,...,...,...,...,...,...
128,Paramount+,Luke Heggie: I Already Told You,Stand-up comedy,"November 11, 2022",NaN,NaN,English,TBA,NaN
129,Paramount+,Kirsty Webeck: Silver Linings,Stand-up comedy,"November 11, 2022",NaN,NaN,English,TBA,NaN
130,Paramount+,Dane Simpson: Didgeridoozy,Stand-up comedy,"November 11, 2022",NaN,NaN,English,TBA,NaN
131,Paramount+,Ivan Aristeguieta: Happy Papi,Stand-up comedy,"November 11, 2022",NaN,NaN,English,TBA,NaN


## Disney+ Data

In [15]:
# Disney+ Data
dis_soup = read_wiki(dis_url, PARAMS)
#print(dis_soup)
dis_genres = get_genres(dis_soup)
#print(dis_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = dis_genres.index("Upcoming original programming")
dis_genres = dis_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
dis_remove = ["Original programming", "Non-English language", "Unscripted", "Specials"]
dis_genres = remove_genres(dis_genres, dis_remove)
#print(dis_genres)
dis_df = get_tables(dis_soup, dis_genres)
#display(dis_df)
dis_df["Language"] = np.NaN
dis_df["Cur. service"] = "Disney+"
dis_df = dis_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime"]]
display(dis_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime
0,Disney+,The Mandalorian,Space western,"November 12, 2019","2 seasons, 16 episodes","Season 3 due to premiere on March 1, 2023[1] R...",NaN,34–55 min
1,Disney+,The Right Stuff,Historical drama,"October 9, 2020","1 season, 8 episodes",Ended[3],NaN,46–53 min
2,Disney+,WandaVision,Superhero romantic comedy-drama,"January 15, 2021",9 episodes,Miniseries,NaN,31–51 min
3,Disney+,The Falcon and the Winter Soldier,Superhero buddy action-adventure,"March 19, 2021",6 episodes,Miniseries,NaN,51–62 min
4,Disney+,Loki,Superhero fantasy action-adventure,"June 9, 2021","1 season, 6 episodes",Season 2 due to premiere in mid-2023[4],NaN,44–56 min
...,...,...,...,...,...,...,...,...
120,Disney+,"The Chorus: Success, Here I Go",Drama/Musical,"September 28, 2022","1 season, 10 episodes",Pending,NaN,37–45 min
121,Disney+,Awaiting release,Awaiting release,Awaiting release,Awaiting release,Awaiting release,NaN,Awaiting release
122,Disney+,Marvel Lucha Libre Edition: El origen de la ma...,Sports/mockumentary,"December 21, 2022",TBA,Pending,NaN,TBA
123,Disney+,Dancing with the Stars (season 31),Reality competition,"September 19, 2022","1 season, 11 episodes",Renewed[34],NaN,115–121 min


## HBO Max Data

In [14]:
# HBO Max Data
hbo_soup = read_wiki(hbo_url, PARAMS)
#print(hbo_soup)
hbo_genres = get_genres(hbo_soup)
#print(hbo_genres)
# Will need to update end based on upcoming genre or stop point 
# based on changes to wiki page
end = hbo_genres.index("Original podcasts")
hbo_genres = hbo_genres[:end]
# Will need to update remove list for headers that don't have tables 
# based on changes to wiki page
hbo_remove = ["Original programming", "Animation", "Non-English language scripted", "Unscripted"]
hbo_genres = remove_genres(hbo_genres, hbo_remove)
#print(hbo_genres)
hbo_df = get_tables(hbo_soup, hbo_genres)
#display(hbo_df)
hbo_df["Cur. service"] = "HBO Max"
hbo_df = hbo_df[["Cur. service", "Title", "Genre", "Premiere", "Seasons", "Status", "Language", "Runtime", "Prev. network(s)"]]
display(hbo_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s)
0,HBO Max,Raised by Wolves,Science fiction,"September 3, 2020","2 seasons, 18 episodes",Ended[2],NaN,39–54 min.,NaN
1,HBO Max,Gossip Girl,Teen drama,"July 8, 2021","2 seasons, 14 episodes",Season 2 ongoing,NaN,53–57 min.,NaN
2,HBO Max,Station Eleven,Post-apocalyptic drama,"December 16, 2021",10 episodes,Miniseries,NaN,44–59 min.,NaN
3,HBO Max,DMZ,Dystopian alternate history drama,"March 17, 2022",4 episodes,Miniseries,NaN,58–60 min.,NaN
4,HBO Max,Julia,Biographical drama,"March 31, 2022","1 season, 8 episodes",Renewed[3],NaN,43–49 min.,NaN
...,...,...,...,...,...,...,...,...,...
135,HBO Max,Friends: The Reunion,Unscripted reunion,"May 27, 2021",NaN,NaN,NaN,"1 hour, 44 min.",NaN
136,HBO Max,Furry Friends Forever: Elmo Gets a Puppy,Animated damily comedy,"August 5, 2021",NaN,NaN,NaN,30 min.,NaN
137,HBO Max,See Us Coming Together: A Sesame Street Special,Family comedy,"November 25, 2021",NaN,NaN,NaN,24 min.,NaN
138,HBO Max,And Just Like That... The Documentary,Making-of,"February 3, 2022",NaN,NaN,NaN,"1 hour, 13 min.",NaN


## Combine Data

In [78]:
# Combine all the dataframes into one dataframe
df_ls = [nflx_cont_df, nflx_end_df, hulu_df, amzn_df, apl_df, prmt_df, dis_df, hbo_df]
all_df = pd.concat(df_ls, axis=0)
display(all_df)

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s)
0,Netflix,Stranger Things,Science fiction horror,"July 15, 2016","4 seasons, 34 episodes",Renewed for final season[1],NaN,42–150 min,NaN
1,Netflix,The Crown,Historical drama,"November 4, 2016","5 seasons, 50 episodes",Renewed for final season[2],NaN,47–61 min,NaN
2,Netflix,The Umbrella Academy,Superhero action,"February 15, 2019","3 seasons, 30 episodes",Renewed for final season[3],NaN,40–60 min,NaN
3,Netflix,Virgin River,Romantic drama,"December 6, 2019","4 seasons, 42 episodes",Renewed[4],NaN,39–49 min,NaN
4,Netflix,The Witcher,Fantasy drama,"December 20, 2019","2 seasons, 16 episodes",Season 3 due to premiere in 2023[5] Renewed fo...,NaN,47–67 min,NaN
...,...,...,...,...,...,...,...,...,...
135,HBO Max,Friends: The Reunion,Unscripted reunion,"May 27, 2021",NaN,NaN,NaN,"1 hour, 44 min.",NaN
136,HBO Max,Furry Friends Forever: Elmo Gets a Puppy,Animated damily comedy,"August 5, 2021",NaN,NaN,NaN,30 min.,NaN
137,HBO Max,See Us Coming Together: A Sesame Street Special,Family comedy,"November 25, 2021",NaN,NaN,NaN,24 min.,NaN
138,HBO Max,And Just Like That... The Documentary,Making-of,"February 3, 2022",NaN,NaN,NaN,"1 hour, 13 min.",NaN


## Clean Data

In [79]:
# Clean all wikipedia data

# Misc fixes
# Remove rows that are empty based on table formatting on the wiki pages
all_df = all_df[all_df["Title"].notnull()]
all_df = all_df[(all_df["Title"] != "Awaiting release") & (all_df["Genre"] != "Awaiting release")]
# Split seasons and episodes into two columns
all_df[['Seasons', 'Episodes']] = all_df['Seasons'].str.split(', ', 1, expand=True)
# Reset index after combining dataframes and removing rows
all_df = all_df.reset_index(drop=True)

for entry in range(0, len(all_df)):
    # Remove citations from Title
    if all_df["Title"][entry] is not np.NaN:
        all_df["Title"][entry] = re.sub("[\[\(].*?[\)\]]", "", all_df["Title"][entry]).rstrip()
    
    # Remove citations from Premiere and convert to date data type
    all_df["Premiere"][entry] = re.sub("[\[\(].*", "", str(all_df["Premiere"][entry])).rstrip()
    if str(all_df["Premiere"][entry]) not in ["nan", "TBA", "2022", "2023", "2024", "Summer 2022", "Mid 2022", "Late 2022"]:
        if bool(re.search("-", all_df["Premiere"][entry])):
            all_df["Premiere"][entry] = dt.strptime(all_df["Premiere"][entry], "%Y-%m-%d %H:%M:%S")
        else:
            all_df["Premiere"][entry] = dt.strptime(all_df["Premiere"][entry], "%B %d, %Y")
    else:
        all_df["Premiere"][entry] = "nan"
    
    # Clean Seasons and Episodes
    # Put miniseries episodes in episodes and mark seasons as 1
    if "episode" in str(all_df["Seasons"][entry]):
        all_df["Episodes"][entry] = all_df["Seasons"][entry]
        all_df["Seasons"][entry] = "1 season"
    # Remove episode label so data is just a number
    all_df["Episodes"][entry] = str(all_df["Episodes"][entry]).split(" ")[0]
    # Mark episodes as unknown
    if all_df["Episodes"][entry] in ["nan", "None", "TBA"]:
        all_df["Episodes"][entry] = "Unknown"
    # Remove season label so data is just a number
    all_df["Seasons"][entry] = str(all_df["Seasons"][entry]).split(" ")[0]
    # Mark seasons as unknown
    if all_df["Seasons"][entry] in ["nan", "TBA"]:
        all_df["Seasons"][entry] = "Unknown"

    # Remove citations from Status and clean data labels
    if all_df["Status"][entry] is not np.NaN:
        no_note = re.sub("\[.*?\]", "", all_df["Status"][entry])
        rename = ["due to premiere", "ongoing", "renewed", "release", "pre-production", "series order", "post-production", "filming"]
        for item in rename:
            if item in no_note.lower():
                replace = "Continuing"
                break
            else:
                replace = no_note
        all_df["Status"][entry] = replace
        
    # Change runtime into buckets (less than 30 min, more than 60, or between 30 and 60) based on median value of range 
    all_df["Runtime"][entry] = re.sub("[\[\(].*?[\)\]]", "", all_df["Runtime"][entry]).rstrip()
    all_df["Runtime"][entry] = re.sub("~", "", all_df["Runtime"][entry]).rstrip()
    if "h" in str(all_df["Runtime"][entry]):
        all_df["Runtime"][entry] = ">60 min"
    else:
        temp = re.sub("[min].*", "", all_df["Runtime"][entry]).rstrip()
        if temp in ["TBA", "na", ""]:
            all_df["Runtime"][entry] = "Unknown"
        else:
            nums = re.split("[\D]", temp)
            nums = [float(x) for x in nums]
            mid = stats.median(nums)
            if mid < 30:
                all_df["Runtime"][entry] = "<30 min"
            elif mid > 60:
                all_df["Runtime"][entry] = ">60 min"
            else:
                all_df["Runtime"][entry] = "<60 min"

C:\Users\kales\AppData\Local\Temp\ipykernel_28400\1920212583.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  all_df[['Seasons', 'Episodes']] = all_df['Seasons'].str.split(', ', 1, expand=True)


## Tidy Data

In [80]:
all_df

,Cur. service,Title,Genre,Premiere,Seasons,Status,Language,Runtime,Prev. network(s),Episodes
0,Netflix,Stranger Things,Science fiction horror,2016-07-15 00:00:00,4,Continuing,NaN,>60 min,NaN,34
1,Netflix,The Crown,Historical drama,2016-11-04 00:00:00,5,Continuing,NaN,<60 min,NaN,50
2,Netflix,The Umbrella Academy,Superhero action,2019-02-15 00:00:00,3,Continuing,NaN,<60 min,NaN,30
3,Netflix,Virgin River,Romantic drama,2019-12-06 00:00:00,4,Continuing,NaN,<60 min,NaN,42
4,Netflix,The Witcher,Fantasy drama,2019-12-20 00:00:00,2,Continuing,NaN,<60 min,NaN,16
...,...,...,...,...,...,...,...,...,...,...
2505,HBO Max,Friends: The Reunion,Unscripted reunion,2021-05-27 00:00:00,Unknown,NaN,NaN,>60 min,NaN,Unknown
2506,HBO Max,Furry Friends Forever: Elmo Gets a Puppy,Animated damily comedy,2021-08-05 00:00:00,Unknown,NaN,NaN,<60 min,NaN,Unknown
2507,HBO Max,See Us Coming Together: A Sesame Street Special,Family comedy,2021-11-25 00:00:00,Unknown,NaN,NaN,<30 min,NaN,Unknown
2508,HBO Max,And Just Like That... The Documentary,Making-of,2022-02-03 00:00:00,Unknown,NaN,NaN,>60 min,NaN,Unknown


## Save Data to CSV

In [81]:
all_df.to_csv("original_tv_streaming.csv")